In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("icustay_detail.csv")

In [3]:
df

,Unnamed: 0,subject_id,hadm_id,icustay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,ethnicity,ethnicity_grouped,hospital_expire_flag,hospstay_seq,first_hosp_stay,intime,outtime,los_icu,icustay_seq,first_icu_stay
0,0,2,163353,243653,M,NaN,2138-07-17 19:04:00,2138-07-21 15:48:00,4,0,ASIAN,asian,0,1,True,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0,1,True
1,1,5,178980,214757,M,NaN,2103-02-02 04:31:00,2103-02-04 12:15:00,2,0,ASIAN,asian,0,1,True,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0,1,True
2,2,110,154943,282073,M,NaN,2110-05-29 23:27:00,2110-06-05 19:12:00,7,0,ASIAN,asian,0,1,True,2110-06-02 02:49:26,2110-06-05 19:47:04,3.0,1,True
3,3,116,127203,208829,M,NaN,2116-10-22 21:26:00,2116-10-26 19:45:00,4,0,ASIAN,asian,0,1,True,2116-10-22 21:29:32,2116-10-23 19:43:32,1.0,1,True
4,4,168,141436,224501,M,NaN,2139-06-11 12:55:00,2139-06-13 11:14:00,2,0,ASIAN,asian,0,1,True,2139-06-11 13:06:33,2139-06-11 20:08:37,0.0,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61046,61046,99559,146121,237309,F,NaN,2153-06-11 21:08:00,2153-06-19 14:40:00,8,89,BLACK/AFRICAN AMERICAN,black,0,1,True,2153-06-15 20:55:50,2153-06-16 15:29:39,1.0,1,True
61047,61047,94351,166930,229802,M,2130-03-15,2130-03-11 15:29:00,2130-03-15 18:00:00,4,89,WHITE - OTHER EUROPEAN,white,1,1,True,2130-03-11 15:30:39,2130-03-15 20:13:40,4.0,1,True
61048,61048,64332,159531,246534,F,2191-03-29,2191-03-27 03:16:00,2191-03-29 17:30:00,2,89,PATIENT DECLINED TO ANSWER,unknown,1,1,True,2191-03-27 03:17:33,2191-03-29 22:41:57,2.0,1,True
61049,61049,79050,135473,262003,F,NaN,2149-10-20 19:34:00,2149-10-23 17:05:00,3,89,HISPANIC/LATINO - PUERTO RICAN,hispanic,0,2,False,2149-10-20 20:26:43,2149-10-21 17:23:10,1.0,1,True


In [5]:
df['admittime'] = pd.to_datetime(df['admittime'])
df['dischtime'] = pd.to_datetime(df['dischtime'])

In [6]:
# Sort by patient and admit time
df = df.sort_values(by=['subject_id', 'admittime'])

# Next admission timestamp and ID
df['NEXT_ADMITTIME'] = df.groupby('subject_id')['admittime'].shift(-1)
df['NEXT_HADM_ID'] = df.groupby('subject_id')['hadm_id'].shift(-1)

# Calculate time gap between current discharge and next admission
df['DAYS_TO_NEXT_ADMIT'] = (df['NEXT_ADMITTIME'] - df['dischtime']).dt.total_seconds() / (3600 * 24)

# Readmission within 30 days (binary target)
df['READMIT_30D'] = df['DAYS_TO_NEXT_ADMIT'].apply(lambda x: 1 if pd.notnull(x) and 0 < x <= 30 else 0)

In [7]:
df

,Unnamed: 0,subject_id,hadm_id,icustay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,...,first_hosp_stay,intime,outtime,los_icu,icustay_seq,first_icu_stay,NEXT_ADMITTIME,NEXT_HADM_ID,DAYS_TO_NEXT_ADMIT,READMIT_30D
0,0,2,163353,243653,M,NaN,2138-07-17 19:04:00,2138-07-21 15:48:00,4,0,...,True,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0,1,True,NaT,NaN,NaN,0
47916,47916,3,145834,211552,M,2102-06-14,2101-10-20 19:08:00,2101-10-31 13:58:00,11,76,...,True,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0,1,True,NaT,NaN,NaN,0
19830,19830,4,185777,294638,F,NaN,2191-03-16 00:28:00,2191-03-23 18:41:00,7,48,...,True,2191-03-16 00:29:31,2191-03-17 16:46:31,1.0,1,True,NaT,NaN,NaN,0
1,1,5,178980,214757,M,NaN,2103-02-02 04:31:00,2103-02-04 12:15:00,2,0,...,True,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0,1,True,NaT,NaN,NaN,0
37046,37046,6,107064,228232,F,NaN,2175-05-30 07:15:00,2175-06-15 16:00:00,16,66,...,True,2175-05-30 21:30:54,2175-06-03 13:39:54,4.0,1,True,NaT,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25294,25294,99985,176670,279638,M,NaN,2181-01-27 02:47:00,2181-02-12 17:05:00,16,54,...,True,2181-01-29 05:33:34,2181-02-09 12:45:20,11.0,1,True,NaT,NaN,NaN,0
19546,19546,99991,151118,226241,M,NaN,2184-12-24 08:30:00,2185-01-05 12:15:00,12,47,...,True,2184-12-28 17:30:58,2184-12-31 20:56:20,3.0,1,True,NaT,NaN,NaN,0
37834,37834,99992,197084,242052,F,NaN,2144-07-25 18:03:00,2144-07-28 17:56:00,3,66,...,True,2144-07-25 18:04:42,2144-07-27 17:27:55,2.0,1,True,NaT,NaN,NaN,0
60992,60992,99995,137810,229633,F,2147-09-29,2147-02-08 08:00:00,2147-02-11 13:15:00,3,89,...,True,2147-02-08 13:53:58,2147-02-10 17:46:30,2.0,1,True,NaT,NaN,NaN,0


In [8]:
df['intime'] = pd.to_datetime(df['intime'])
df['outtime'] = pd.to_datetime(df['outtime'])

In [9]:
# Sort by patient and admit time
df = df.sort_values(by=['subject_id', 'intime'])

# Next admission timestamp and ID
df['NEXT_ADMITTIME_ICU'] = df.groupby('subject_id')['intime'].shift(-1)
df['NEXT_HADM_ID_ICU'] = df.groupby('subject_id')['hadm_id'].shift(-1)

# Calculate time gap between current discharge and next admission
df['DAYS_TO_NEXT_ADMIT_ICU'] = (df['NEXT_ADMITTIME_ICU'] - df['outtime']).dt.total_seconds() / (3600 * 24)

# Readmission within 30 days (binary target)
df['READMIT_30D_ICU'] = df['DAYS_TO_NEXT_ADMIT_ICU'].apply(lambda x: 1 if pd.notnull(x) and 0 < x <= 30 else 0)

In [10]:
df

,Unnamed: 0,subject_id,hadm_id,icustay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,...,icustay_seq,first_icu_stay,NEXT_ADMITTIME,NEXT_HADM_ID,DAYS_TO_NEXT_ADMIT,READMIT_30D,NEXT_ADMITTIME_ICU,NEXT_HADM_ID_ICU,DAYS_TO_NEXT_ADMIT_ICU,READMIT_30D_ICU
0,0,2,163353,243653,M,NaN,2138-07-17 19:04:00,2138-07-21 15:48:00,4,0,...,1,True,NaT,NaN,NaN,0,NaT,NaN,NaN,0
47916,47916,3,145834,211552,M,2102-06-14,2101-10-20 19:08:00,2101-10-31 13:58:00,11,76,...,1,True,NaT,NaN,NaN,0,NaT,NaN,NaN,0
19830,19830,4,185777,294638,F,NaN,2191-03-16 00:28:00,2191-03-23 18:41:00,7,48,...,1,True,NaT,NaN,NaN,0,NaT,NaN,NaN,0
1,1,5,178980,214757,M,NaN,2103-02-02 04:31:00,2103-02-04 12:15:00,2,0,...,1,True,NaT,NaN,NaN,0,NaT,NaN,NaN,0
37046,37046,6,107064,228232,F,NaN,2175-05-30 07:15:00,2175-06-15 16:00:00,16,66,...,1,True,NaT,NaN,NaN,0,NaT,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25294,25294,99985,176670,279638,M,NaN,2181-01-27 02:47:00,2181-02-12 17:05:00,16,54,...,1,True,NaT,NaN,NaN,0,NaT,NaN,NaN,0
19546,19546,99991,151118,226241,M,NaN,2184-12-24 08:30:00,2185-01-05 12:15:00,12,47,...,1,True,NaT,NaN,NaN,0,NaT,NaN,NaN,0
37834,37834,99992,197084,242052,F,NaN,2144-07-25 18:03:00,2144-07-28 17:56:00,3,66,...,1,True,NaT,NaN,NaN,0,NaT,NaN,NaN,0
60992,60992,99995,137810,229633,F,2147-09-29,2147-02-08 08:00:00,2147-02-11 13:15:00,3,89,...,1,True,NaT,NaN,NaN,0,NaT,NaN,NaN,0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61051 entries, 0 to 34618
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              61051 non-null  int64         
 1   subject_id              61051 non-null  int64         
 2   hadm_id                 61051 non-null  int64         
 3   icustay_id              61051 non-null  int64         
 4   gender                  61051 non-null  object        
 5   dod                     23889 non-null  object        
 6   admittime               61051 non-null  datetime64[ns]
 7   dischtime               61051 non-null  datetime64[ns]
 8   los_hospital            61051 non-null  int64         
 9   admission_age           61051 non-null  int64         
 10  ethnicity               61051 non-null  object        
 11  ethnicity_grouped       61051 non-null  object        
 12  hospital_expire_flag    61051 non-null  int64      

In [19]:
df.drop(columns=["admittime", "dischtime", "intime", "outtime", "ethnicity", "dod"], inplace=True)

In [20]:
df

,Unnamed: 0,subject_id,hadm_id,icustay_id,gender,los_hospital,admission_age,ethnicity_grouped,hospital_expire_flag,hospstay_seq,first_hosp_stay,los_icu,icustay_seq,first_icu_stay,READMIT_30D,READMIT_30D_ICU
0,0,2,163353,243653,M,4,0,asian,0,1,True,0.0,1,True,0,0
47916,47916,3,145834,211552,M,11,76,white,0,1,True,6.0,1,True,0,0
19830,19830,4,185777,294638,F,7,48,white,0,1,True,1.0,1,True,0,0
1,1,5,178980,214757,M,2,0,asian,0,1,True,0.0,1,True,0,0
37046,37046,6,107064,228232,F,16,66,white,0,1,True,4.0,1,True,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25294,25294,99985,176670,279638,M,16,54,white,0,1,True,11.0,1,True,0,0
19546,19546,99991,151118,226241,M,12,47,white,0,1,True,3.0,1,True,0,0
37834,37834,99992,197084,242052,F,3,66,white,0,1,True,2.0,1,True,0,0
60992,60992,99995,137810,229633,F,3,89,white,0,1,True,2.0,1,True,0,0


In [21]:
# Select columns for one-hot encoding
columns_to_encode = ['ethnicity_grouped', 'first_hosp_stay', 'first_icu_stay']

In [22]:
# Perform one-hot encoding using pd.get_dummies()
df_encoded = pd.get_dummies(df, columns=columns_to_encode, drop_first=True)

In [23]:
# Display the encoded DataFrame
df_encoded.head()

,Unnamed: 0,subject_id,hadm_id,icustay_id,gender,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,...,READMIT_30D,READMIT_30D_ICU,ethnicity_grouped_black,ethnicity_grouped_hispanic,ethnicity_grouped_native,ethnicity_grouped_other,ethnicity_grouped_unknown,ethnicity_grouped_white,first_hosp_stay_True,first_icu_stay_True
0,0,2,163353,243653,M,4,0,0,1,0.0,...,0,0,False,False,False,False,False,False,True,True
47916,47916,3,145834,211552,M,11,76,0,1,6.0,...,0,0,False,False,False,False,False,True,True,True
19830,19830,4,185777,294638,F,7,48,0,1,1.0,...,0,0,False,False,False,False,False,True,True,True
1,1,5,178980,214757,M,2,0,0,1,0.0,...,0,0,False,False,False,False,False,False,True,True
37046,37046,6,107064,228232,F,16,66,0,1,4.0,...,0,0,False,False,False,False,False,True,True,True


In [25]:
df.columns

Index(['Unnamed: 0', 'subject_id', 'hadm_id', 'icustay_id', 'gender',
       'los_hospital', 'admission_age', 'ethnicity_grouped',
       'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'los_icu',
       'icustay_seq', 'first_icu_stay', 'READMIT_30D', 'READMIT_30D_ICU'],
      dtype='object')

In [ ]:
df.drop(columns=["ethnicity_grouped_black", "dischtime", "intime", "outtime", "ethnicity", "dod"], inplace=True)

In [26]:
# Convert categorical columns to category type
df['ethnicity_grouped'] = df['ethnicity_grouped'].astype('category')
df['first_hosp_stay'] = df['first_hosp_stay'].astype('category')
df['first_icu_stay'] = df['first_icu_stay'].astype('category')

# Encode the categories as integer labels
df['ethnicity_grouped_encoded'] = df['ethnicity_grouped'].cat.codes
df['first_hosp_stay_encoded'] = df['first_hosp_stay'].cat.codes
df['first_icu_stay_encoded'] = df['first_icu_stay'].cat.codes

# Drop the original columns if you want
df.drop(['ethnicity_grouped', 'first_hosp_stay', 'first_icu_stay'], axis=1, inplace=True)

In [27]:
df

,Unnamed: 0,subject_id,hadm_id,icustay_id,gender,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,READMIT_30D,READMIT_30D_ICU,ethnicity_grouped_encoded,first_hosp_stay_encoded,first_icu_stay_encoded
0,0,2,163353,243653,M,4,0,0,1,0.0,1,0,0,0,1,1
47916,47916,3,145834,211552,M,11,76,0,1,6.0,1,0,0,6,1,1
19830,19830,4,185777,294638,F,7,48,0,1,1.0,1,0,0,6,1,1
1,1,5,178980,214757,M,2,0,0,1,0.0,1,0,0,0,1,1
37046,37046,6,107064,228232,F,16,66,0,1,4.0,1,0,0,6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25294,25294,99985,176670,279638,M,16,54,0,1,11.0,1,0,0,6,1,1
19546,19546,99991,151118,226241,M,12,47,0,1,3.0,1,0,0,6,1,1
37834,37834,99992,197084,242052,F,3,66,0,1,2.0,1,0,0,6,1,1
60992,60992,99995,137810,229633,F,3,89,0,1,2.0,1,0,0,6,1,1


In [28]:
# Binary Encoding
df['gender_encoded'] = df['gender'].apply(lambda x: 1 if x == 'M' else 0)

In [29]:
df

,Unnamed: 0,subject_id,hadm_id,icustay_id,gender,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,READMIT_30D,READMIT_30D_ICU,ethnicity_grouped_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,gender_encoded
0,0,2,163353,243653,M,4,0,0,1,0.0,1,0,0,0,1,1,1
47916,47916,3,145834,211552,M,11,76,0,1,6.0,1,0,0,6,1,1,1
19830,19830,4,185777,294638,F,7,48,0,1,1.0,1,0,0,6,1,1,0
1,1,5,178980,214757,M,2,0,0,1,0.0,1,0,0,0,1,1,1
37046,37046,6,107064,228232,F,16,66,0,1,4.0,1,0,0,6,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25294,25294,99985,176670,279638,M,16,54,0,1,11.0,1,0,0,6,1,1,1
19546,19546,99991,151118,226241,M,12,47,0,1,3.0,1,0,0,6,1,1,1
37834,37834,99992,197084,242052,F,3,66,0,1,2.0,1,0,0,6,1,1,0
60992,60992,99995,137810,229633,F,3,89,0,1,2.0,1,0,0,6,1,1,0


In [30]:
df.drop(['gender'], axis=1, inplace=True)

In [31]:
df.to_csv("icustay_detail_encoded.csv")

In [32]:
df1 = pd.read_csv("apsii.csv")

In [33]:
df1

,Unnamed: 0,subject_id,hadm_id,icustay_id,apsiii,apsiii_prob,hr_score,meanbp_score,temp_score,resprate_score,...,wbc_score,creatinine_score,uo_score,bun_score,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score
0,0,11216,188044,200120,0,0.011705,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3367,149879,200202,0,0.011705,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,9736,106592,200213,0,0.011705,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,17450,174819,200260,0,0.011705,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,16994,115444,200493,0,0.011705,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,61527,8498,100336,249814,137,0.884793,7.0,23.0,16.0,18.0,...,0.0,10.0,15.0,7.0,2.0,6.0,0.0,3.0,12.0,0.0
61528,61528,25030,102205,255030,139,0.894083,17.0,23.0,2.0,18.0,...,5.0,10.0,15.0,11.0,2.0,0.0,0.0,3.0,1.0,24.0
61529,61529,56264,135431,275638,140,0.898476,5.0,15.0,NaN,18.0,...,0.0,10.0,15.0,11.0,0.0,0.0,0.0,3.0,12.0,48.0
61530,61530,3222,197362,275746,140,0.898476,7.0,15.0,20.0,18.0,...,0.0,7.0,7.0,11.0,4.0,NaN,NaN,0.0,0.0,48.0


In [39]:
df1 = df1[["subject_id", "hadm_id", "icustay_id", "apsiii_prob"]]

In [40]:
df1

,subject_id,hadm_id,icustay_id,apsiii_prob
0,11216,188044,200120,0.011705
1,3367,149879,200202,0.011705
2,9736,106592,200213,0.011705
3,17450,174819,200260,0.011705
4,16994,115444,200493,0.011705
...,...,...,...,...
61527,8498,100336,249814,0.884793
61528,25030,102205,255030,0.894083
61529,56264,135431,275638,0.898476
61530,3222,197362,275746,0.898476


In [41]:
df1.to_csv("apsii_filtered.csv")

In [3]:
df2 = pd.read_csv("mloads.csv")

In [4]:
df2

,Unnamed: 0,icustay_id,mLODS,neurologic,cardiovascular,renal,pulmonary,hematologic,hepatic
0,0,200020,0,NaN,NaN,NaN,0,0.0,NaN
1,1,200023,0,NaN,0.0,NaN,0,0.0,NaN
2,2,200120,0,NaN,NaN,NaN,0,NaN,NaN
3,3,200195,0,NaN,NaN,NaN,0,0.0,NaN
4,4,200202,0,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
61527,61527,282152,16,5.0,5.0,3.0,3,0.0,0.0
61528,61528,287041,16,5.0,3.0,3.0,3,1.0,1.0
61529,61529,296988,16,5.0,3.0,3.0,3,1.0,1.0
61530,61530,223926,17,5.0,5.0,3.0,3,1.0,0.0


In [5]:
df2[["icustay_id","mLODS"]]

,icustay_id,mLODS
0,200020,0
1,200023,0
2,200120,0
3,200195,0
4,200202,0
...,...,...
61527,282152,16
61528,287041,16
61529,296988,16
61530,223926,17


In [7]:
df2.to_csv("mloads_filtered.csv")

In [8]:
df3 = pd.read_csv("sirs.csv")

In [9]:
df3

,Unnamed: 0,subject_id,hadm_id,icustay_id,sirs,temp_score,heartrate_score,resp_score,wbc_score
0,0,11216,188044,200120,0,NaN,NaN,NaN,NaN
1,1,24635,180673,200187,0,NaN,NaN,0.0,0.0
2,2,3367,149879,200202,0,NaN,NaN,NaN,NaN
3,3,9736,106592,200213,0,NaN,NaN,NaN,NaN
4,4,17450,174819,200260,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
61527,61527,27219,102678,299969,4,1.0,1.0,1.0,1.0
61528,61528,17532,129682,299981,4,1.0,1.0,1.0,1.0
61529,61529,11776,161707,299988,4,1.0,1.0,1.0,1.0
61530,61530,13083,186679,299992,4,1.0,1.0,1.0,1.0


In [10]:
df3[["subject_id", "hadm_id", "icustay_id", "sirs"]]

,subject_id,hadm_id,icustay_id,sirs
0,11216,188044,200120,0
1,24635,180673,200187,0
2,3367,149879,200202,0
3,9736,106592,200213,0
4,17450,174819,200260,0
...,...,...,...,...
61527,27219,102678,299969,4
61528,17532,129682,299981,4
61529,11776,161707,299988,4
61530,13083,186679,299992,4


In [11]:
df3.to_csv("sirs_filtered.csv")

In [12]:
df4 = pd.read_csv("sofa.csv")

In [13]:
df4

,Unnamed: 0,subject_id,hadm_id,icustay_id,SOFA,respiration,coagulation,liver,cardiovascular,cns,renal
0,0,20948,109512,200018,0,NaN,0.0,NaN,NaN,NaN,NaN
1,1,17020,158088,200020,0,NaN,0.0,NaN,NaN,NaN,NaN
2,2,4638,145058,200023,0,NaN,0.0,NaN,NaN,NaN,NaN
3,3,11216,188044,200120,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,23569,134773,200121,0,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
61527,61527,15275,114904,231258,21,3.0,4.0,2.0,4.0,4.0,4.0
61528,61528,21515,117562,263045,21,3.0,4.0,3.0,4.0,4.0,3.0
61529,61529,30184,164766,287041,21,4.0,3.0,2.0,4.0,4.0,4.0
61530,61530,77893,160969,289980,21,4.0,4.0,2.0,4.0,4.0,3.0


In [15]:
df4[["subject_id", "hadm_id", "icustay_id", "SOFA"]]

,subject_id,hadm_id,icustay_id,SOFA
0,20948,109512,200018,0
1,17020,158088,200020,0
2,4638,145058,200023,0
3,11216,188044,200120,0
4,23569,134773,200121,0
...,...,...,...,...
61527,15275,114904,231258,21
61528,21515,117562,263045,21
61529,30184,164766,287041,21
61530,77893,160969,289980,21


In [16]:
df4.to_csv("sofa_filtered.csv")

In [18]:
df5 = pd.read_csv("sapsii.csv")
df5

,Unnamed: 0,subject_id,hadm_id,icustay_id,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,temp_score,...,uo_score,bun_score,wbc_score,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score
0,0,23295,188810,231141,0,0.000425,0,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0
1,1,50939,182020,243336,0,0.000425,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2,15374,114732,273982,0,0.000425,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,3,67987,100019,296288,0,0.000425,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,4,24635,180673,200187,6,0.004584,0,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,61527,55849,113883,293830,101,0.986509,18,11.0,13.0,0.0,...,11.0,6.0,0.0,0.0,1.0,0.0,NaN,26.0,0,6
61528,61528,97351,158101,260758,103,0.988558,18,11.0,5.0,0.0,...,11.0,6.0,3.0,3.0,0.0,3.0,0.0,26.0,0,6
61529,61529,27965,171782,295509,103,0.988558,18,11.0,13.0,0.0,...,11.0,10.0,0.0,3.0,0.0,5.0,0.0,26.0,0,6
61530,61530,53432,120167,225233,106,0.991061,7,11.0,13.0,0.0,...,4.0,6.0,12.0,3.0,5.0,3.0,9.0,7.0,9,6


In [43]:
df5[["subject_id", "hadm_id", "icustay_id", "sapsii_prob"]]

,subject_id,hadm_id,icustay_id,sapsii_prob
0,23295,188810,231141,0.000425
1,50939,182020,243336,0.000425
2,15374,114732,273982,0.000425
3,67987,100019,296288,0.000425
4,24635,180673,200187,0.004584
...,...,...,...,...
61527,55849,113883,293830,0.986509
61528,97351,158101,260758,0.988558
61529,27965,171782,295509,0.988558
61530,53432,120167,225233,0.991061


In [40]:
df5.to_csv("sapsii_filtered.csv")

In [21]:
df6 = pd.read_csv("suspicion_of_infection.csv")

In [22]:
df6

,Unnamed: 0,icustay_id,antibiotic_name,antibiotic_time,last72_charttime,next24_charttime,suspected_infection_time,specimen,positiveculture
0,0,216423,Cipro,2163-04-24,2163-04-21 03:00:00,2163-04-24 03:30:00,2163-04-21 03:00:00,URINE,0.0
1,1,216423,Cipro,2163-04-24,2163-04-21 03:00:00,2163-04-24 15:00:00,2163-04-21 03:00:00,URINE,0.0
2,2,235733,Cipro,2124-09-09,2124-09-08 00:00:00,2124-09-09 17:00:00,2124-09-08 00:00:00,URINE,0.0
3,3,270663,Cipro,2106-01-05,2106-01-05 00:00:00,2106-01-05 00:00:00,2106-01-05 00:00:00,URINE,0.0
4,4,272582,Cipro,2124-09-09,2124-09-08 00:00:00,2124-09-09 17:00:00,2124-09-08 00:00:00,URINE,0.0
...,...,...,...,...,...,...,...,...,...
1374907,1374907,206308,Erythromycin Ethylsuccinate Suspension,2126-11-28,2126-11-26 03:14:00,NaN,2126-11-26 03:14:00,URINE,0.0
1374908,1374908,204737,Erythromycin Ethylsuccinate Suspension,2193-08-14,2193-08-13 14:00:00,NaN,2193-08-13 14:00:00,BLOOD CULTURE,0.0
1374909,1374909,266909,Erythromycin Ethylsuccinate Suspension,2124-12-19,2124-12-17 09:02:00,NaN,2124-12-17 09:02:00,URINE,0.0
1374910,1374910,296275,Erythromycin Ethylsuccinate Suspension,2110-10-28,2110-10-25 13:00:00,NaN,2110-10-25 13:00:00,BRONCHOALVEOLAR LAVAGE,1.0


In [23]:
df = df6.copy()

In [24]:
# Feature 1: Count of antibiotics administered per ICU stay
antibiotic_counts = df.groupby('icustay_id')['antibiotic_name'].count().reset_index()
antibiotic_counts.columns = ['icustay_id', 'antibiotic_count']

# Feature 2: Unique antibiotics administered per ICU stay
unique_antibiotics = df.groupby('icustay_id')['antibiotic_name'].nunique().reset_index()
unique_antibiotics.columns = ['icustay_id', 'n_unique_antibiotics']

# Feature 3: Number of positive cultures per ICU stay
positive_cultures = df.groupby('icustay_id')['positiveculture'].sum().reset_index()
positive_cultures.columns = ['icustay_id', 'positive_culture_count']

# Feature 4: Specimen Type Count
specimen_counts = df.groupby(['icustay_id', 'specimen']).size().unstack(fill_value=0).reset_index()

# Feature 5: Time-based Features
df['antibiotic_time'] = pd.to_datetime(df['antibiotic_time'])
df['last72_charttime'] = pd.to_datetime(df['last72_charttime'])
df['next24_charttime'] = pd.to_datetime(df['next24_charttime'])

# Calculate duration of antibiotic treatment
treatment_duration = df.groupby('icustay_id').agg({
    'antibiotic_time': ['min', 'max']
}).reset_index()
treatment_duration.columns = ['icustay_id', 'first_antibiotic_time', 'last_antibiotic_time']
treatment_duration['antibiotic_duration'] = (treatment_duration['last_antibiotic_time'] - treatment_duration['first_antibiotic_time']).dt.total_seconds() / 3600  # in hours

# Merging all features into a single DataFrame
features = antibiotic_counts.merge(unique_antibiotics, on='icustay_id', how='left')
features = features.merge(positive_cultures, on='icustay_id', how='left')
features = features.merge(specimen_counts, on='icustay_id', how='left')
features = features.merge(treatment_duration[['icustay_id', 'antibiotic_duration']], on='icustay_id', how='left')

# Display the processed features DataFrame
features.head()

,icustay_id,antibiotic_count,n_unique_antibiotics,positive_culture_count,ABSCESS,ANORECTAL/VAGINAL CULTURE,ARTHROPOD,ASPIRATE,BILE,BIOPSY,...,Touch Prep/Sections,URINE,"URINE,KIDNEY","URINE,SUPRAPUBIC ASPIRATE",VARICELLA-ZOSTER CULTURE,VIRAL CULTURE: R/O CYTOMEGALOVIRUS,VIRAL CULTURE:R/O HERPES SIMPLEX VIRUS,WORM,XXX,antibiotic_duration
0,200001,88,2,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0
1,200003,87,8,22.0,0.0,0.0,0.0,0.0,9.0,0.0,...,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,480.0
2,200006,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200007,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200009,3,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0


In [25]:
features.columns

Index(['icustay_id', 'antibiotic_count', 'n_unique_antibiotics',
       'positive_culture_count', 'ABSCESS', 'ANORECTAL/VAGINAL CULTURE',
       'ARTHROPOD', 'ASPIRATE', 'BILE', 'BIOPSY', 'BLOOD', 'BLOOD CULTURE',
       'BLOOD CULTURE ( MYCO/F LYTIC BOTTLE)', 'BLOOD CULTURE - NEONATE',
       'BONE MARROW', 'BONE MARROW - CYTOGENETICS', 'BRONCHIAL BRUSH',
       'BRONCHIAL BRUSH - PROTECTED', 'BRONCHIAL WASHINGS',
       'BRONCHOALVEOLAR LAVAGE', 'Blood (CMV AB)', 'Blood (EBV)',
       'Blood (Malaria)', 'Blood (Toxo)', 'C, E, & A Screening',
       'CATHETER TIP-IV', 'CORNEAL EYE SCRAPINGS', 'CRE Screen',
       'CSF;SPINAL FLUID', 'DIALYSIS FLUID',
       'DIRECT ANTIGEN TEST FOR VARICELLA-ZOSTER VIRUS',
       'Direct Antigen Test for Herpes Simplex Virus Types 1 & 2', 'EAR',
       'EYE', 'FECAL SWAB', 'FLUID RECEIVED IN BLOOD CULTURE BOTTLES',
       'FLUID WOUND', 'FLUID,OTHER', 'FOOT CULTURE', 'FOREIGN BODY',
       'GASTRIC ASPIRATE', 'IMMUNOLOGY', 'Immunology (CMV)',
       '

In [41]:
data1 = pd.read_csv("apsii_filtered.csv").drop(columns=['Unnamed: 0'], errors='ignore')
data2 = pd.read_csv("mloads_filtered.csv").drop(columns=['Unnamed: 0'], errors='ignore')
data3 = pd.read_csv("sapsii_filtered.csv").drop(columns=['Unnamed: 0'], errors='ignore')
data4 = pd.read_csv("sirs_filtered.csv").drop(columns=['Unnamed: 0'], errors='ignore')
data5 = pd.read_csv("sofa_filtered.csv").drop(columns=['Unnamed: 0'], errors='ignore')

In [31]:
data1

,subject_id,hadm_id,icustay_id,apsiii_prob
0,11216,188044,200120,0.011705
1,3367,149879,200202,0.011705
2,9736,106592,200213,0.011705
3,17450,174819,200260,0.011705
4,16994,115444,200493,0.011705
...,...,...,...,...
61527,8498,100336,249814,0.884793
61528,25030,102205,255030,0.894083
61529,56264,135431,275638,0.898476
61530,3222,197362,275746,0.898476


In [33]:
data2 = pd.read_csv("mloads_filtered.csv").drop(columns=['Unnamed: 0.1'], errors='ignore')

In [35]:
data2.columns

Index(['Unnamed: 0', 'icustay_id', 'mLODS', 'neurologic', 'cardiovascular',
       'renal', 'pulmonary', 'hematologic', 'hepatic'],
      dtype='object')

In [36]:
data2.drop(columns=['Unnamed: 0'], errors='ignore')

,icustay_id,mLODS,neurologic,cardiovascular,renal,pulmonary,hematologic,hepatic
0,200020,0,NaN,NaN,NaN,0,0.0,NaN
1,200023,0,NaN,0.0,NaN,0,0.0,NaN
2,200120,0,NaN,NaN,NaN,0,NaN,NaN
3,200195,0,NaN,NaN,NaN,0,0.0,NaN
4,200202,0,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...
61527,282152,16,5.0,5.0,3.0,3,0.0,0.0
61528,287041,16,5.0,3.0,3.0,3,1.0,1.0
61529,296988,16,5.0,3.0,3.0,3,1.0,1.0
61530,223926,17,5.0,5.0,3.0,3,1.0,0.0


In [52]:
data3 = data3[["subject_id", "hadm_id", "icustay_id", "sapsii_prob"]]
data3

,subject_id,hadm_id,icustay_id,sapsii_prob
0,23295,188810,231141,0.000425
1,50939,182020,243336,0.000425
2,15374,114732,273982,0.000425
3,67987,100019,296288,0.000425
4,24635,180673,200187,0.004584
...,...,...,...,...
61527,55849,113883,293830,0.986509
61528,97351,158101,260758,0.988558
61529,27965,171782,295509,0.988558
61530,53432,120167,225233,0.991061


In [45]:
data4

,Unnamed: 0.1,subject_id,hadm_id,icustay_id,sirs,temp_score,heartrate_score,resp_score,wbc_score
0,0,11216,188044,200120,0,NaN,NaN,NaN,NaN
1,1,24635,180673,200187,0,NaN,NaN,0.0,0.0
2,2,3367,149879,200202,0,NaN,NaN,NaN,NaN
3,3,9736,106592,200213,0,NaN,NaN,NaN,NaN
4,4,17450,174819,200260,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
61527,61527,27219,102678,299969,4,1.0,1.0,1.0,1.0
61528,61528,17532,129682,299981,4,1.0,1.0,1.0,1.0
61529,61529,11776,161707,299988,4,1.0,1.0,1.0,1.0
61530,61530,13083,186679,299992,4,1.0,1.0,1.0,1.0


In [46]:
data4.drop(columns=['Unnamed: 0.1'], errors='ignore')

,subject_id,hadm_id,icustay_id,sirs,temp_score,heartrate_score,resp_score,wbc_score
0,11216,188044,200120,0,NaN,NaN,NaN,NaN
1,24635,180673,200187,0,NaN,NaN,0.0,0.0
2,3367,149879,200202,0,NaN,NaN,NaN,NaN
3,9736,106592,200213,0,NaN,NaN,NaN,NaN
4,17450,174819,200260,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
61527,27219,102678,299969,4,1.0,1.0,1.0,1.0
61528,17532,129682,299981,4,1.0,1.0,1.0,1.0
61529,11776,161707,299988,4,1.0,1.0,1.0,1.0
61530,13083,186679,299992,4,1.0,1.0,1.0,1.0


In [49]:
data4 = data4[["subject_id", "hadm_id", "icustay_id", "sirs"]]

In [50]:
data4

,subject_id,hadm_id,icustay_id,sirs
0,11216,188044,200120,0
1,24635,180673,200187,0
2,3367,149879,200202,0
3,9736,106592,200213,0
4,17450,174819,200260,0
...,...,...,...,...
61527,27219,102678,299969,4
61528,17532,129682,299981,4
61529,11776,161707,299988,4
61530,13083,186679,299992,4


In [53]:
data5

,Unnamed: 0.1,subject_id,hadm_id,icustay_id,SOFA,respiration,coagulation,liver,cardiovascular,cns,renal
0,0,20948,109512,200018,0,NaN,0.0,NaN,NaN,NaN,NaN
1,1,17020,158088,200020,0,NaN,0.0,NaN,NaN,NaN,NaN
2,2,4638,145058,200023,0,NaN,0.0,NaN,NaN,NaN,NaN
3,3,11216,188044,200120,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,23569,134773,200121,0,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
61527,61527,15275,114904,231258,21,3.0,4.0,2.0,4.0,4.0,4.0
61528,61528,21515,117562,263045,21,3.0,4.0,3.0,4.0,4.0,3.0
61529,61529,30184,164766,287041,21,4.0,3.0,2.0,4.0,4.0,4.0
61530,61530,77893,160969,289980,21,4.0,4.0,2.0,4.0,4.0,3.0


In [55]:
data5.drop(columns=['Unnamed: 0.1'], errors='ignore')

,subject_id,hadm_id,icustay_id,SOFA,respiration,coagulation,liver,cardiovascular,cns,renal
0,20948,109512,200018,0,NaN,0.0,NaN,NaN,NaN,NaN
1,17020,158088,200020,0,NaN,0.0,NaN,NaN,NaN,NaN
2,4638,145058,200023,0,NaN,0.0,NaN,NaN,NaN,NaN
3,11216,188044,200120,0,NaN,NaN,NaN,NaN,NaN,NaN
4,23569,134773,200121,0,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
61527,15275,114904,231258,21,3.0,4.0,2.0,4.0,4.0,4.0
61528,21515,117562,263045,21,3.0,4.0,3.0,4.0,4.0,3.0
61529,30184,164766,287041,21,4.0,3.0,2.0,4.0,4.0,4.0
61530,77893,160969,289980,21,4.0,4.0,2.0,4.0,4.0,3.0


In [56]:
data5 = data5[["subject_id", "hadm_id", "icustay_id", "SOFA"]]

In [57]:
# Store DataFrames in a dictionary for easier handling
datasets = {
    "data1": data1,
    "data2": data2,
    "data3": data3,
    "data4": data4,
    "data5": data5
}

# Identify available keys in each DataFrame
keys_in_datasets = {}
for name, df in datasets.items():
    available_keys = [key for key in ['subject_id', 'hadm_id', 'icustay_id'] if key in df.columns]
    keys_in_datasets[name] = available_keys
    print(f"{name} keys: {available_keys}")

# Initialize merged_df with the most complete DataFrame (having all keys if possible)
merged_df = data1

# Merge datasets progressively based on available keys
for name, df in list(datasets.items())[1:]:
    merge_keys = keys_in_datasets[name]
    if merge_keys:
        merged_df = pd.merge(merged_df, df, on=merge_keys, how='inner')
    else:
        print(f"Skipping {name} as it has no matching keys.")

# Display the merged DataFrame
print(merged_df.head())

data1 keys: ['subject_id', 'hadm_id', 'icustay_id']
data2 keys: ['icustay_id']
data3 keys: ['subject_id', 'hadm_id', 'icustay_id']
data4 keys: ['subject_id', 'hadm_id', 'icustay_id']
data5 keys: ['subject_id', 'hadm_id', 'icustay_id']
   subject_id  hadm_id  icustay_id  apsiii_prob  Unnamed: 0.1  mLODS  \
0       11216   188044      200120     0.011705             2      0   
1        3367   149879      200202     0.011705             4      0   
2        9736   106592      200213     0.011705             5      0   
3       17450   174819      200260     0.011705             7      0   
4       16994   115444      200493     0.011705            12      0   

   neurologic  cardiovascular  renal  pulmonary  hematologic  hepatic  \
0         NaN             NaN    NaN          0          NaN      NaN   
1         NaN             NaN    NaN          0          NaN      NaN   
2         NaN             NaN    NaN          0          NaN      NaN   
3         NaN             NaN    NaN    

In [58]:
merged_df.drop(columns=['Unnamed: 0.1'], errors='ignore')

,subject_id,hadm_id,icustay_id,apsiii_prob,mLODS,neurologic,cardiovascular,renal,pulmonary,hematologic,hepatic,sapsii_prob,sirs,SOFA
0,11216,188044,200120,0.011705,0,NaN,NaN,NaN,0,NaN,NaN,0.037205,0,0
1,3367,149879,200202,0.011705,0,NaN,NaN,NaN,0,NaN,NaN,0.004584,0,0
2,9736,106592,200213,0.011705,0,NaN,NaN,NaN,0,NaN,NaN,0.037205,0,0
3,17450,174819,200260,0.011705,0,NaN,NaN,NaN,0,NaN,NaN,0.004584,0,0
4,16994,115444,200493,0.011705,0,NaN,NaN,NaN,0,NaN,NaN,0.029295,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,8498,100336,249814,0.884793,9,0.0,3.0,3.0,3,0.0,0.0,0.905156,4,13
61528,25030,102205,255030,0.894083,7,3.0,1.0,3.0,0,0.0,0.0,0.660981,4,11
61529,56264,135431,275638,0.898476,12,5.0,1.0,3.0,3,0.0,0.0,0.949924,2,16
61530,3222,197362,275746,0.898476,9,5.0,1.0,3.0,0,0.0,NaN,0.897459,3,10


In [59]:
merged_df.to_csv("scores.csv")

In [60]:
scores = pd.read_csv("scores.csv")
scores

,Unnamed: 0,subject_id,hadm_id,icustay_id,apsiii_prob,Unnamed: 0.1,mLODS,neurologic,cardiovascular,renal,pulmonary,hematologic,hepatic,sapsii_prob,sirs,SOFA
0,0,11216,188044,200120,0.011705,2,0,NaN,NaN,NaN,0,NaN,NaN,0.037205,0,0
1,1,3367,149879,200202,0.011705,4,0,NaN,NaN,NaN,0,NaN,NaN,0.004584,0,0
2,2,9736,106592,200213,0.011705,5,0,NaN,NaN,NaN,0,NaN,NaN,0.037205,0,0
3,3,17450,174819,200260,0.011705,7,0,NaN,NaN,NaN,0,NaN,NaN,0.004584,0,0
4,4,16994,115444,200493,0.011705,12,0,NaN,NaN,NaN,0,NaN,NaN,0.029295,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,61527,8498,100336,249814,0.884793,60556,9,0.0,3.0,3.0,3,0.0,0.0,0.905156,4,13
61528,61528,25030,102205,255030,0.894083,59132,7,3.0,1.0,3.0,0,0.0,0.0,0.660981,4,11
61529,61529,56264,135431,275638,0.898476,61382,12,5.0,1.0,3.0,3,0.0,0.0,0.949924,2,16
61530,61530,3222,197362,275746,0.898476,60675,9,5.0,1.0,3.0,0,0.0,NaN,0.897459,3,10


In [15]:
icu = pd.read_csv("icustay_detail_encoded.csv")

In [4]:
icu

,Unnamed: 0.1,Unnamed: 0,subject_id,hadm_id,icustay_id,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,READMIT_30D,READMIT_30D_ICU,ethnicity_grouped_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,gender_encoded
0,0,0,2,163353,243653,4,0,0,1,0.0,1,0,0,0,1,1,1
1,47916,47916,3,145834,211552,11,76,0,1,6.0,1,0,0,6,1,1,1
2,19830,19830,4,185777,294638,7,48,0,1,1.0,1,0,0,6,1,1,0
3,1,1,5,178980,214757,2,0,0,1,0.0,1,0,0,0,1,1,1
4,37046,37046,6,107064,228232,16,66,0,1,4.0,1,0,0,6,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61046,25294,25294,99985,176670,279638,16,54,0,1,11.0,1,0,0,6,1,1,1
61047,19546,19546,99991,151118,226241,12,47,0,1,3.0,1,0,0,6,1,1,1
61048,37834,37834,99992,197084,242052,3,66,0,1,2.0,1,0,0,6,1,1,0
61049,60992,60992,99995,137810,229633,3,89,0,1,2.0,1,0,0,6,1,1,0


In [5]:
icu = icu.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], errors='ignore')

In [6]:
icu

,subject_id,hadm_id,icustay_id,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,READMIT_30D,READMIT_30D_ICU,ethnicity_grouped_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,gender_encoded
0,2,163353,243653,4,0,0,1,0.0,1,0,0,0,1,1,1
1,3,145834,211552,11,76,0,1,6.0,1,0,0,6,1,1,1
2,4,185777,294638,7,48,0,1,1.0,1,0,0,6,1,1,0
3,5,178980,214757,2,0,0,1,0.0,1,0,0,0,1,1,1
4,6,107064,228232,16,66,0,1,4.0,1,0,0,6,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61046,99985,176670,279638,16,54,0,1,11.0,1,0,0,6,1,1,1
61047,99991,151118,226241,12,47,0,1,3.0,1,0,0,6,1,1,1
61048,99992,197084,242052,3,66,0,1,2.0,1,0,0,6,1,1,0
61049,99995,137810,229633,3,89,0,1,2.0,1,0,0,6,1,1,0


In [7]:
scores = pd.read_csv("scores.csv")

In [8]:
scores

,Unnamed: 0,subject_id,hadm_id,icustay_id,apsiii_prob,Unnamed: 0.1,mLODS,neurologic,cardiovascular,renal,pulmonary,hematologic,hepatic,sapsii_prob,sirs,SOFA
0,0,11216,188044,200120,0.011705,2,0,NaN,NaN,NaN,0,NaN,NaN,0.037205,0,0
1,1,3367,149879,200202,0.011705,4,0,NaN,NaN,NaN,0,NaN,NaN,0.004584,0,0
2,2,9736,106592,200213,0.011705,5,0,NaN,NaN,NaN,0,NaN,NaN,0.037205,0,0
3,3,17450,174819,200260,0.011705,7,0,NaN,NaN,NaN,0,NaN,NaN,0.004584,0,0
4,4,16994,115444,200493,0.011705,12,0,NaN,NaN,NaN,0,NaN,NaN,0.029295,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,61527,8498,100336,249814,0.884793,60556,9,0.0,3.0,3.0,3,0.0,0.0,0.905156,4,13
61528,61528,25030,102205,255030,0.894083,59132,7,3.0,1.0,3.0,0,0.0,0.0,0.660981,4,11
61529,61529,56264,135431,275638,0.898476,61382,12,5.0,1.0,3.0,3,0.0,0.0,0.949924,2,16
61530,61530,3222,197362,275746,0.898476,60675,9,5.0,1.0,3.0,0,0.0,NaN,0.897459,3,10


In [9]:
scores.columns

Index(['Unnamed: 0', 'subject_id', 'hadm_id', 'icustay_id', 'apsiii_prob',
       'Unnamed: 0.1', 'mLODS', 'neurologic', 'cardiovascular', 'renal',
       'pulmonary', 'hematologic', 'hepatic', 'sapsii_prob', 'sirs', 'SOFA'],
      dtype='object')

In [10]:
scores = scores.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'neurologic', 'cardiovascular', 'renal',
       'pulmonary', 'hematologic', 'hepatic'], errors='ignore')

In [11]:
# Perform an inner join on the common columns
final_df = pd.merge(icu, scores, on=['subject_id', 'hadm_id', 'icustay_id'], how='inner')

# Display the merged DataFrame
print(final_df.head())
print(f"Number of rows after merge: {final_df.shape[0]}")

   subject_id  hadm_id  icustay_id  los_hospital  admission_age  \
0           2   163353      243653             4              0   
1           3   145834      211552            11             76   
2           4   185777      294638             7             48   
3           5   178980      214757             2              0   
4           6   107064      228232            16             66   

   hospital_expire_flag  hospstay_seq  los_icu  icustay_seq  READMIT_30D  \
0                     0             1      0.0            1            0   
1                     0             1      6.0            1            0   
2                     0             1      1.0            1            0   
3                     0             1      0.0            1            0   
4                     0             1      4.0            1            0   

   READMIT_30D_ICU  ethnicity_grouped_encoded  first_hosp_stay_encoded  \
0                0                          0                     

In [12]:
final_df.to_csv("ICU_scores.csv")

In [13]:
final_df.head()

,subject_id,hadm_id,icustay_id,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,READMIT_30D,READMIT_30D_ICU,ethnicity_grouped_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,gender_encoded,apsiii_prob,mLODS,sapsii_prob,sirs,SOFA
0,2,163353,243653,4,0,0,1,0.0,1,0,0,0,1,1,1,0.050995,4,0.046739,2,4
1,3,145834,211552,11,76,0,1,6.0,1,0,0,6,1,1,1,0.341819,7,0.918976,3,7
2,4,185777,294638,7,48,0,1,1.0,1,0,0,6,1,1,0,0.048756,0,0.166523,2,4
3,5,178980,214757,2,0,0,1,0.0,1,0,0,0,1,1,1,0.011705,0,0.004584,1,0
4,6,107064,228232,16,66,0,1,4.0,1,0,0,6,1,1,0,0.090356,3,0.058153,1,4


In [14]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61051 entries, 0 to 61050
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   subject_id                 61051 non-null  int64  
 1   hadm_id                    61051 non-null  int64  
 2   icustay_id                 61051 non-null  int64  
 3   los_hospital               61051 non-null  int64  
 4   admission_age              61051 non-null  int64  
 5   hospital_expire_flag       61051 non-null  int64  
 6   hospstay_seq               61051 non-null  int64  
 7   los_icu                    61041 non-null  float64
 8   icustay_seq                61051 non-null  int64  
 9   READMIT_30D                61051 non-null  int64  
 10  READMIT_30D_ICU            61051 non-null  int64  
 11  ethnicity_grouped_encoded  61051 non-null  int64  
 12  first_hosp_stay_encoded    61051 non-null  int64  
 13  first_icu_stay_encoded     61051 non-null  int

In [3]:
oasis =pd.read_csv("oasis.csv")

In [2]:
import pandas as pd

In [4]:
oasis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61532 entries, 0 to 61531
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             61532 non-null  int64  
 1   subject_id             61532 non-null  int64  
 2   hadm_id                61532 non-null  int64  
 3   icustay_id             61532 non-null  int64  
 4   icustay_age_group      61532 non-null  object 
 5   hospital_expire_flag   61532 non-null  int64  
 6   icustay_expire_flag    61532 non-null  int64  
 7   oasis                  61532 non-null  int64  
 8   oasis_PROB             61532 non-null  float64
 9   age                    61532 non-null  int64  
 10  age_score              61532 non-null  int64  
 11  preiculos              61532 non-null  int64  
 12  preiculos_score        61532 non-null  int64  
 13  gcs                    52143 non-null  float64
 14  gcs_score              52143 non-null  float64
 15  he

In [5]:
oasis.columns

Index(['Unnamed: 0', 'subject_id', 'hadm_id', 'icustay_id',
       'icustay_age_group', 'hospital_expire_flag', 'icustay_expire_flag',
       'oasis', 'oasis_PROB', 'age', 'age_score', 'preiculos',
       'preiculos_score', 'gcs', 'gcs_score', 'heartrate', 'heartrate_score',
       'meanbp', 'meanbp_score', 'resprate', 'resprate_score', 'temp',
       'temp_score', 'urineoutput', 'urineoutput_score', 'mechvent',
       'mechvent_score', 'electivesurgery', 'electivesurgery_score'],
      dtype='object')

In [6]:
oasis= oasis[['subject_id', 'hadm_id', 'icustay_id','oasis_PROB']]

In [7]:
oasis

,subject_id,hadm_id,icustay_id,oasis_PROB
0,17450,174819,200260,0.004454
1,23485,190581,201216,0.004454
2,4037,178771,201471,0.004454
3,23279,102032,201534,0.004454
4,20228,140627,202763,0.004454
...,...,...,...,...
61527,93898,132413,211785,0.772134
61528,95706,169554,231714,0.793786
61529,16013,113853,212312,0.832423
61530,31573,148606,221625,0.832423


In [8]:
oasis.to_csv("oasis.csv")

In [9]:
icu_scores =pd.read_csv("ICU_scores.csv")

In [10]:
# Perform an inner join on the common columns
final = pd.merge(icu_scores, oasis, on=['subject_id', 'hadm_id', 'icustay_id'], how='inner')

# Display the merged DataFrame
print(final.head())
print(f"Number of rows after merge: {final.shape[0]}")

   Unnamed: 0  subject_id  hadm_id  icustay_id  los_hospital  admission_age  \
0           0           2   163353      243653             4              0   
1           1           3   145834      211552            11             76   
2           2           4   185777      294638             7             48   
3           3           5   178980      214757             2              0   
4           4           6   107064      228232            16             66   

   hospital_expire_flag  hospstay_seq  los_icu  icustay_seq  ...  \
0                     0             1      0.0            1  ...   
1                     0             1      6.0            1  ...   
2                     0             1      1.0            1  ...   
3                     0             1      0.0            1  ...   
4                     0             1      4.0            1  ...   

   ethnicity_grouped_encoded  first_hosp_stay_encoded  first_icu_stay_encoded  \
0                          0       

In [11]:
final = final.drop(columns=['Unnamed: 0'], errors='ignore')

In [12]:
final

,subject_id,hadm_id,icustay_id,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,READMIT_30D,...,ethnicity_grouped_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,gender_encoded,apsiii_prob,mLODS,sapsii_prob,sirs,SOFA,oasis_PROB
0,2,163353,243653,4,0,0,1,0.0,1,0,...,0,1,1,1,0.050995,4,0.046739,2,4,0.013897
1,3,145834,211552,11,76,0,1,6.0,1,0,...,6,1,1,1,0.341819,7,0.918976,3,7,0.748927
2,4,185777,294638,7,48,0,1,1.0,1,0,...,6,1,1,0,0.048756,0,0.166523,2,4,0.054187
3,5,178980,214757,2,0,0,1,0.0,1,0,...,0,1,1,1,0.011705,0,0.004584,1,0,0.006515
4,6,107064,228232,16,66,0,1,4.0,1,0,...,6,1,1,0,0.090356,3,0.058153,1,4,0.008392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61046,99985,176670,279638,16,54,0,1,11.0,1,0,...,6,1,1,1,0.181550,4,0.140051,4,5,0.254529
61047,99991,151118,226241,12,47,0,1,3.0,1,0,...,6,1,1,1,0.075975,3,0.017445,3,4,0.029395
61048,99992,197084,242052,3,66,0,1,2.0,1,0,...,6,1,1,0,0.116527,2,0.152870,1,3,0.042509
61049,99995,137810,229633,3,89,0,1,2.0,1,0,...,6,1,1,0,0.107143,2,0.071716,2,5,0.048012


In [14]:
final.to_csv("ICU_scores.csv")

In [17]:
elixhauser_score = pd.read_csv("elixhauser_score.csv")

In [18]:
elixhauser_score

,Unnamed: 0,hadm_id,elixhauser_vanwalraven,elixhauser_SID29,elixhauser_SID30
0,0,113124,NaN,NaN,NaN
1,1,116945,NaN,NaN,NaN
2,2,117414,NaN,NaN,NaN
3,3,121288,NaN,NaN,NaN
4,4,144088,NaN,NaN,NaN
...,...,...,...,...,...
58971,58971,160738,-1.0,-1.0,-1.0
58972,58972,151811,-1.0,0.0,-1.0
58973,58973,122990,-1.0,0.0,-1.0
58974,58974,133483,-1.0,-1.0,-1.0


In [19]:
elixhauser_score = elixhauser_score[['hadm_id', 'elixhauser_SID30']]

In [20]:
# Perform an inner join on the common columns
final = pd.merge(icu_scores, elixhauser_score, on=['hadm_id'], how='inner')

# Display the merged DataFrame
print(final.head())
print(f"Number of rows after merge: {final.shape[0]}")

   Unnamed: 0  subject_id  hadm_id  icustay_id  los_hospital  admission_age  \
0           0           2   163353      243653             4              0   
1           1           3   145834      211552            11             76   
2           2           4   185777      294638             7             48   
3           3           5   178980      214757             2              0   
4           4           6   107064      228232            16             66   

   hospital_expire_flag  hospstay_seq  los_icu  icustay_seq  ...  \
0                     0             1      0.0            1  ...   
1                     0             1      6.0            1  ...   
2                     0             1      1.0            1  ...   
3                     0             1      0.0            1  ...   
4                     0             1      4.0            1  ...   

   ethnicity_grouped_encoded  first_hosp_stay_encoded  first_icu_stay_encoded  \
0                          0       

In [21]:
final = final.drop(columns=['Unnamed: 0'], errors='ignore')

In [22]:
final

,subject_id,hadm_id,icustay_id,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,READMIT_30D,...,ethnicity_grouped_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,gender_encoded,apsiii_prob,mLODS,sapsii_prob,sirs,SOFA,elixhauser_SID30
0,2,163353,243653,4,0,0,1,0.0,1,0,...,0,1,1,1,0.050995,4,0.046739,2,4,0.0
1,3,145834,211552,11,76,0,1,6.0,1,0,...,6,1,1,1,0.341819,7,0.918976,3,7,19.0
2,4,185777,294638,7,48,0,1,1.0,1,0,...,6,1,1,0,0.048756,0,0.166523,2,4,28.0
3,5,178980,214757,2,0,0,1,0.0,1,0,...,0,1,1,1,0.011705,0,0.004584,1,0,0.0
4,6,107064,228232,16,66,0,1,4.0,1,0,...,6,1,1,0,0.090356,3,0.058153,1,4,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61046,99985,176670,279638,16,54,0,1,11.0,1,0,...,6,1,1,1,0.181550,4,0.140051,4,5,0.0
61047,99991,151118,226241,12,47,0,1,3.0,1,0,...,6,1,1,1,0.075975,3,0.017445,3,4,18.0
61048,99992,197084,242052,3,66,0,1,2.0,1,0,...,6,1,1,0,0.116527,2,0.152870,1,3,-7.0
61049,99995,137810,229633,3,89,0,1,2.0,1,0,...,6,1,1,0,0.107143,2,0.071716,2,5,13.0


In [23]:
final.to_csv("ICU_scores.csv")

In [24]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61051 entries, 0 to 61050
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   subject_id                 61051 non-null  int64  
 1   hadm_id                    61051 non-null  int64  
 2   icustay_id                 61051 non-null  int64  
 3   los_hospital               61051 non-null  int64  
 4   admission_age              61051 non-null  int64  
 5   hospital_expire_flag       61051 non-null  int64  
 6   hospstay_seq               61051 non-null  int64  
 7   los_icu                    61041 non-null  float64
 8   icustay_seq                61051 non-null  int64  
 9   READMIT_30D                61051 non-null  int64  
 10  READMIT_30D_ICU            61051 non-null  int64  
 11  ethnicity_grouped_encoded  61051 non-null  int64  
 12  first_hosp_stay_encoded    61051 non-null  int64  
 13  first_icu_stay_encoded     61051 non-null  int

In [25]:
 # Emergency Visits (E) - Combining hospstay_seq and icustay_seq into an integer
final['combined_seq'] = (final['hospstay_seq'] * 1000) + final['icustay_seq']

In [27]:
final = final.sort_values(by=['subject_id', 'hadm_id'])

# Calculate Previous Emergency Visits using cumcount() correctly
final['prev_emergency_visits'] = final.groupby('subject_id').cumcount()

# Display the updated DataFrame
print(final[['subject_id', 'hadm_id', 'icustay_id', 'hospstay_seq', 'icustay_seq', 'prev_emergency_visits']].head(20))

    subject_id  hadm_id  icustay_id  hospstay_seq  icustay_seq  \
0            2   163353      243653             1            1   
1            3   145834      211552             1            1   
2            4   185777      294638             1            1   
3            5   178980      214757             1            1   
4            6   107064      228232             1            1   
5            7   118037      278444             1            1   
6            7   118037      236754             1            2   
7            8   159514      262299             1            1   
8            9   150750      220597             1            1   
9           10   184167      288409             1            1   
10          11   194540      229441             1            1   
11          12   112213      232669             1            1   
12          13   143045      263738             1            1   
13          16   103251      240232             1            1   
15        

In [5]:
df = pd.read_csv("ICU_scores.csv")

In [6]:
readmit_df = df[df['READMIT_30D'] == 1]

In [7]:
readmit_counts = readmit_df['subject_id'].value_counts()

In [8]:
readmit_counts.value_counts().sort_index()

count
1     1920
2      290
3       77
4       25
5       11
6        7
8        2
9        2
10       1
13       1
21       1
26       1
Name: count, dtype: int64